# Hyper parameter tunning
- updated 0912
- data  
5th_FE_ver3_before   
5th_FE_ver3  

- catboost 에만 label encoding data 사용한다.

- ML basic 모델들 튜닝
- catboost 튜닝

## y_predict 값들 피클로 저장해놓기

## ready

In [ ]:
cd /content/drive/My Drive/빅콘

/content/drive/My Drive/빅콘


In [ ]:
!apt-get update --qq
!apt-get install fonts-nanum* -qq

E: Command line option --qq is not understood in combination with the other options
Selecting previously unselected package fonts-nanum.
(Reading database ... 144579 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig 

In [ ]:
# 한글 폰트
import matplotlib.font_manager as fm
fm._rebuild()

In [ ]:
# # CUDA
# !pip install mxnet-cu80
# import mxnet as mx

In [ ]:
# ready
import os
import sys
import joblib
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd 
import numpy as np
import random
import math

from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_info_columns', 500)

import imp
import datetime
from tqdm import tqdm

In [ ]:
# vusualize
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

In [ ]:
# sklearn
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score

## Load Data
- 5th_FE_ver3_before  
- 5th_FE_ver3


data['X'] / data['y']

### 5th_FE_ver3

In [ ]:
# load data
## ver3
v3 = joblib.load(os.path.join('data', '5th_FE_ver3.pkl'))
locals().update(v3)

v3_label = joblib.load(os.path.join('data', '5th_FE_ver3_before.pkl'))
locals().update(v3_label)

In [ ]:
# X, y 설정
X = v3['X'] ; y = v3['y'] ; log_y = np.log(y)
X_label = v3_label['X']

In [ ]:
X.isnull().sum().sum(), X_label.isnull().sum().sum()

(267028, 267028)

In [ ]:
# fill na
X = X.fillna(0); X_label = X_label.fillna(0)

# One Hot
X_dum = pd.get_dummies(X)
X.shape, X_dum.shape, X_label.shape

((35379, 274), (35379, 274), (35379, 216))

### Cross Validation function

In [ ]:
# mape
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# mape with exp
def MAPE_exp(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((np.exp(y_true) - np.exp(y_pred)) / np.exp(y_true))) * 100

In [ ]:
# MAPE_exp_cv 구현
def MAPE_exp_cv5(model,X,y,cv_splits):        
    # setting
    mape = []
    # for 문 돌리기
    cv=KFold(n_splits=cv_splits,random_state=42,shuffle=True)
    for t,v in cv.split(X):
        X_train=X.iloc[t]       # 훈련용
        X_val=X.iloc[v]         # 검증용 분리.
        y_train=y.iloc[t]
        y_val=y.iloc[v]

        # model
        model.fit(X_train,y_train)
        y_pred = model.predict(X_val)
        #y_pred_list.append(np.exp(y_pred))
        # mape
        mape.append(MAPE_exp(y_val, y_pred))
    return mape, np.mean(mape), y_pred_list

In [ ]:
# stacking with cv=5
# MAPE_exp_cv 구현

def MAPE_exp_cv5_st(model,X,y,cv_splits):        
    # setting
    mape = []
    pred_list = []     # X_val_ 으로 예측
    test_list = []     # X_test 으로 예측
    # split train, test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=77)

    # for 문 돌리기
    cv=KFold(n_splits=cv_splits,random_state=77,shuffle=True)
    for t,v in cv.split(X_train):
        X_train_ = X_train.iloc[t]       # 훈련용
        X_val_ = X_train.iloc[v]         # 검증용 분리.
        y_train_ = y_train.iloc[t]
        y_val_ = y_train.iloc[v]

        # model
        model.fit(X_train_,y_train_)
        y_pred = model.predict(X_val_)
        pred_list.append(np.exp(y_pred))
        
        test_pred = model.predict(X_test)
        test_list.append(np.exp(test_pred))
        # mape
        mape.append(MAPE_exp(y_val_, y_pred))
    return mape, np.mean(mape), pred_list, test_list 

### Modeling with cv

### CatBoost
예측해야하는 값이 훈련 셋 피처에 들어가버리는 문제, 즉 Data Leakage 문제  
doc : https://catboost.ai/docs/concepts/python-reference_utils_eval_metric.html  
kaggle : https://www.kaggle.com/mitribunskiy/tutorial-catboost-overview
  
    
mape : 31.306861591528538

In [ ]:
!pip install catboost

     |████████████████████████████████| 66.1MB 60kB/s 


In [ ]:
from catboost import CatBoostRegressor

In [ ]:
# catboost setting
cb_model = CatBoostRegressor(iterations=5000,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='MAPE',
                             random_seed = 77,
                             bagging_temperature = 0.2,
                             early_stopping_rounds = 500,
                             #group_id = []
                             od_type='Iter',
                             metric_period = 50,     # ?
                             #od_wait=20,
                             thread_count=-1)

In [ ]:
print('catboost : \n',MAPE_exp_cv5_st(cb_model,X_dum,log_y,5))

평균 mape 32.73414280484671  
arraty 5개씩 * 2 개 잘나옴

### tunning

In [ ]:
cat_features = ['상품군','결제방법','조기매진','isHoliday',
                '방송월','방송일','방송시간(시간)','방송시간(분)',
                '평일여부','방송시간대','계절','분기','성별',
                '상품군_가격대','전체_가격대'
                ]

In [ ]:
categorical_features_indices = np.where(X_label.dtypes != np.float)[0];categorical_features_indices

array([  1,   2,   3,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,
        17,  29,  30,  32,  80,  81,  82,  83,  84,  85,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109])

In [ ]:
# Setting X, y
X = X_label
y = log_y

# params
param_grid = {
'learning_rate':[0.01,0.1,0.05],
'min_data_in_leaf': range(10,20,2),
'depth': range(5,30,2),
'border_count': [0.1,0.2,0.3,0.4,0.5,0.6],
'bagging_temperature' : [0.1,0.2,0.3,0.5]
}

params = {}
params['iterations'] = 5000
#params['custom_loss'] = 'MAPE'
params['eval_metric'] = 'MAPE'
params['random_seed'] = 77
# 튜닝 내용
#params['learning_rate'] = learning_rate
# params['min_data_in_leaf'] = int(round(min_data_in_leaf))    
# params['depth'] = int(round(depth))         
# #params['max_leaves'] = int(round(max_leaves))
# #params['l2_leaf_reg'] = int(round(l2_leaf_reg))
# params['border_count'] = int(round(border_count))
# params['bagging_temperature'] = int(round(bagging_temperature))

# model
cb_model = CatBoostRegressor(**params ,
                             early_stopping_rounds = 500,
                             #task_type = "GPU",
                             leaf_estimation_iterations = 10,
                             #use_best_model=True,
                             od_type="Iter",
                             #logging_level='Silent',
    )
    
# fit
cb_model.fit(X, y, 
             cat_features=categorical_features_indices,
             #eval_set=(X_val_, y_val_), 
             verbose = 1000)

# param_grid = {
# 'learning_rate':[0.01,0.1,0.05],
# 'min_data_in_leaf': range(10,20,2),
# 'depth': range(5,30,2),
# 'border_count': [0.1,0.2,0.3,0.4,0.5,0.6],
# 'baggiing_temperature' : [0.1,0.2,0.3,0.5]
# }


# define grid_search
grid_search = GridSearchCV(estimator=cb_model, 
                        param_grid=param_grid,  
                        n_jobs=-1, 
                        verbose=2
                        )

# fit with (x_train, y_train)
grid_search.fit(X, y)

In [ ]:
# best params
grid_search.best_params_

In [ ]:
### 변수 중요도 시각화
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
cat_feature_imp = pd.DataFrame([CatBoost.feature_names_ , CatBoost.feature_importances_]).T
cat_feature_imp.columns = ["feature","varimp"]
cat_feature_imp = cat_feature_imp.sort_values(by="varimp", ascending = False)
sns.barplot(y="feature", x="varimp",data = cat_feature_imp, )
plt.show()

## baysyan 기반
https://data-newbie.tistory.com/159

In [ ]:
! pip install bayesian-optimization

In [ ]:
from bayes_opt import BayesianOptimization

In [ ]:
def catboost_eval(bagging_temperature ,
                  depth , 
                  learning_rate ,
                  min_data_in_leaf ,  
                  border_count):
    n_splits=5
    cv = KFold(n_splits=5, shuffle=True, random_state=77)
    mape = []
    predict = None
    params = {}
    params['iterations'] = 5000
    #params['custom_loss'] = 'MAPE'
    params['eval_metric'] = 'MAPE'
    params['random_seed'] = 1234
    params['learning_rate'] = learning_rate
    params['min_data_in_leaf'] = int(round(min_data_in_leaf))
    params['depth'] = int(round(depth))
    #params['max_leaves'] = int(round(max_leaves))
    #params['l2_leaf_reg'] = int(round(l2_leaf_reg))
    params['border_count'] = int(round(border_count))
    params['bagging_temperature'] = int(round(bagging_temperature))
    
    X, y = X_dum, log_y 

    for tr_ind, val_ind in cv.split(X , y):
        X_train = X.iloc[tr_ind]
        y_train = y.iloc[tr_ind]
        X_valid = X.iloc[val_ind]
        y_valid = y.iloc[val_ind]
        ## https://catboost.ai/docs/concepts/python-reference_catboost_eval-metrics.html
        model = CatBoostRegressor(**params , 
                                 #task_type = "GPU" , 
                                 leaf_estimation_iterations = 10,
                                 use_best_model=True,
                                 od_type="Iter",
                                 #logging_level='Silent',
                                )
        model.fit(X_train, 
                y_train,
                #cat_features=categorical_features_indices,
                eval_set=(X_valid, y_valid),
                verbose = False ,
        )
        
        y_pred = model.predict(X_valid)
        mape.append(MAPE_exp(y_pred,y_valid))
    return -np.mean(mape)

## min_data_in_leaf , max_leaves , 추가 하니 먼가 잘 안됨.
catBO = BayesianOptimization(catboost_eval,
                             {'bagging_temperature': (0, 1000),
                              'depth': (5, 8.99) ,
                              "learning_rate" : (0.001,0.1) , 
                              "min_data_in_leaf" : (1,6) , 
                              #'max_leaves' : (200,200)  ,
                              #'l2_leaf_reg': (100, 100)  ,
                              'border_count': (5, 255) ,
                             },
                             random_state=0)
# init_round=5
# opt_round = 10
catBO.maximize(init_points=10, n_iter=20)

### 변수 중요도 시각화
# import seaborn as sns
# sns.set(rc={'figure.figsize':(11.7,8.27)})
# cat_feature_imp = pd.DataFrame([CatBoost.feature_names_ , CatBoost.feature_importances_]).T
# cat_feature_imp.columns = ["feature","varimp"]
# cat_feature_imp = cat_feature_imp.sort_values(by="varimp", ascending = False)
# sns.barplot(y="feature", x="varimp",data = cat_feature_imp, )
# plt.show()

|   iter    |  target   | baggin... | border... |   depth   | learni... | min_da... |
-------------------------------------------------------------------------------------
|  1        | -29.34    |  548.8    |  183.8    |  7.405    |  0.05494  |  3.118    |
|  2        | -29.52    |  645.9    |  114.4    |  8.558    |  0.0964   |  2.917    |
|  3        | -29.5     |  791.7    |  137.2    |  7.266    |  0.09263  |  1.355    |
|  4        | -29.36    |  87.13    |  10.05    |  8.322    |  0.07804  |  5.35     |
|  5        | -29.3     |  978.6    |  204.8    |  6.841    |  0.07827  |  1.591    |
|  6        | -29.13    |  639.9    |  40.84    |  8.769    |  0.05266  |  3.073    |
|  7        | -29.29    |  264.6    |  198.6    |  6.82     |  0.05727  |  1.094    |
|  8        | -29.58    |  617.6    |  158.0    |  7.462    |  0.09443  |  4.409    |
|  9        | -30.8     |  359.5    |  114.3    |  7.784    |  0.006962 |  4.334    |
|  10       | -29.52    |  670.6    |  57.6     |  5.5


|   iter    |  target   | baggin... | border... |   depth   | learni... | min_da... |  
|  6        | -29.13    |  639.9    |  40.84    |  8.769    |  0.05266  |  3.073   
|27       | -28.91    |  86.21    |  8.92     |  8.834    |  0.0124   |  5.818    |  
  

In [ ]:
catBO.max

{'params': {'bagging_temperature': 86.20566967662191,
  'border_count': 8.920475731018419,
  'depth': 8.834206374187707,
  'learning_rate': 0.012401575319148032,
  'min_data_in_leaf': 5.817638932445064},
 'target': -28.91074826341997}

In [ ]:
cb_model = CatBoostRegressor(iterations = 100000,
                             learning_rate=0.012401575319148032,
                             depth=int(round(8.834206374187707)),     # 8.834,
                             eval_metric='MAPE',
                             random_seed = 77,
                             bagging_temperature = int(round(86.20566967662191)),
                             early_stopping_rounds = 500,
                             border_count = int(round(8.920475731018419)),     # 8.92
                             min_data_in_leaf = int(round(5.817638932445064)),
                             od_type='Iter',
                             verbose = 5000,
                             #metric_period = 50,     # ?
                             thread_count=-1)
                             

In [ ]:
print('catboost : \n',MAPE_exp_cv5_st(cb_model,X_dum,log_y,5))

0:	learn: 0.0487482	total: 17.9ms	remaining: 29m 50s
5000:	learn: 0.0098463	total: 1m 27s	remaining: 27m 34s
10000:	learn: 0.0055585	total: 2m 52s	remaining: 25m 56s
15000:	learn: 0.0033890	total: 4m 18s	remaining: 24m 27s
20000:	learn: 0.0021573	total: 5m 44s	remaining: 22m 59s
25000:	learn: 0.0014129	total: 7m 10s	remaining: 21m 32s
30000:	learn: 0.0009470	total: 8m 36s	remaining: 20m 4s
35000:	learn: 0.0006445	total: 10m 1s	remaining: 18m 37s
40000:	learn: 0.0004450	total: 11m 26s	remaining: 17m 10s
45000:	learn: 0.0003122	total: 12m 51s	remaining: 15m 43s
50000:	learn: 0.0002223	total: 14m 16s	remaining: 14m 16s
55000:	learn: 0.0001606	total: 15m 42s	remaining: 12m 51s
60000:	learn: 0.0001178	total: 17m 8s	remaining: 11m 25s
65000:	learn: 0.0000876	total: 18m 33s	remaining: 9m 59s
70000:	learn: 0.0000667	total: 19m 59s	remaining: 8m 33s
75000:	learn: 0.0000516	total: 21m 25s	remaining: 7m 8s
80000:	learn: 0.0000410	total: 22m 50s	remaining: 5m 42s
85000:	learn: 0.0000336	total: 24m

iter -> 10000, mape 32.64249999009495